# Data Preparation and Feature Engineering

In this demo, we'll delve into techniques such as preparing modeling data, including splitting data, handling missing values, encoding categorical features, and standardizing features. We will also discuss outlier removal and coercing columns to the correct data type.

By the end, you will have a comprehensive understanding of data preparation for modeling and feature preparation.

---

## Learning Objectives:

By the end of this demo, you will be able to:

- Coerce columns to be the correct data type based on feature or target variable type.
- Identify and remove outliers from the modeling data.
- Drop rows/columns that contain missing values.
- Impute categorical missing values with the mode value.
- Replace missing values with a specified replacement value.
- One-hot encode categorical features.
- Apply pre-existing embeddings to categorical features.
- Standardize features in a training set.

## Classroom Setup

Before starting the demo, run the provided classroom setup script. This script will define configuration variables necessary for the demo. Execute the following cell:


In [0]:
# Classroom Setup
%run ../Includes/Classroom-Setup-01

# Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
# Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


## Other Conventions:

Throughout this demo, we'll refer to the object `DA`. This object, provided by Databricks Academy, contains variables such as your username, catalog name, schema name, working directory, and dataset locations. Run the code block below to view these details:


In [0]:
print(f"Username: {DA.username}")
print(f"Catalog Name: {DA.catalog_name}")
print(f"Schema Name: {DA.schema_name}")
print(f"Working Directory: {DA.paths.working_dir}")
print(f"Dataset Location: {DA.paths.datasets}")


## Data Cleaning and Imputation

- Load the dataset from the specified path using Spark and read it as a DataFrame.
- Drop any rows with missing values from the DataFrame using the `dropna()` method.
- Fill any remaining missing values in the DataFrame with the 0 using the `fillna()` method.
- Create a temporary view named as `telco_customer_churn`


In [0]:
dataset_path = f"{DA.paths.datasets}/telco/telco-customer-churn-noisy.csv"
telco_df = spark.read.csv(dataset_path, header="true", inferSchema="true", multiLine="true", escape='"')

# telco_df.printSchema()
display(telco_df)


### Coerce/Fix Data Types

Even though most of the data types are correct, let's do the following to have a better memory footprint of the dataframe in memory:

- Convert `SeniorCitizen` and `Churn` binary columns to boolean type.
- Converting the `tenure` column to a long integer using `.selectExpr` and reordering the columns.
- Using `spark.sql` to convert `Partner`, `Dependents`, `PhoneService` and `PaperlessBilling` columns to boolean, and reordering the columns. Then, saving the dataframe as a DELTA table.


In [0]:
from pyspark.sql.types import BooleanType, ShortType, IntegerType
from pyspark.sql.functions import col, when

binary_columns = ["SeniorCitizen", "Churn"]
telco_customer_churn_df = telco_df

for column in binary_columns:
    telco_customer_churn_df = telco_customer_churn_df.withColumn(column, col(column).cast(BooleanType()))

telco_customer_churn_df.select(*binary_columns).printSchema()


Casting didn't work on `SeniorCitizen` most probably because there were some null values or values which couldn't be encoded correctly. We can force coercion using a simple filter method (assuming missing values in this column can be encoded as `False`).


In [0]:
telco_customer_churn_df = telco_customer_churn_df.withColumn(
    "SeniorCitizen", when(col("SeniorCitizen") == 1, True).otherwise(False)
)

telco_customer_churn_df.select("SeniorCitizen").printSchema()


In [0]:
telco_customer_churn_df.head()

In [0]:
# PhoneService & PaperlessBilling to new boolean using spark.sql and re-order columns
telco_customer_churn_df.createOrReplaceTempView("telco_customer_churn_temp_view")

telco_customer_casted_df = spark.sql("""
  SELECT
    customerID,
    BOOLEAN(Dependents),
    BOOLEAN(Partner),
    BOOLEAN(PhoneService),
    BOOLEAN(PaperlessBilling),
    *
  EXCEPT (customerID, Dependents, Partner, PhoneService, PaperlessBilling, Churn),
    Churn
  FROM telco_customer_churn_temp_view
""")

telco_customer_casted_df.select("Dependents", "Partner", "PaperlessBilling", "PhoneService").printSchema()


In [0]:
# Tenure months to Long/Integer using .selectExpr
telco_customer_casted_df = telco_customer_churn_df.selectExpr("* except(tenure)", "cast(tenure as long) tenure")
telco_customer_casted_df.select("tenure").printSchema()


In [0]:
telco_customer_name_full = "telco_customer_full"

# [OPTIONAL] Save as DELTA table (silver)
telco_customer_full_silver = f"{telco_customer_name_full}_silver"
telco_customer_casted_df.write.mode("overwrite").option("mergeSchema", True).saveAsTable(telco_customer_full_silver)

# print(telco_customer_casted_df)


### Handling Outliers

We will see how to handle outliers in column by identifying and addressing data points that fall far outside the typical range of values in a dataset. Common methods for handling outliers include removing them, filtering, transforming the data, or replacing outliers with more representative values.

Follow these steps for handling outliers:

- Create a new silver table named as `telco_customer_full_silver` by appending `silver` to the original table name and then accessing it using Spark SQL.
- Filtering out outliers from the `TotalCharges` column by removing rows where the column value exceeds the specified cutoff value.


Filtering out outliers from the `TotalCharges` column by removing rows where the column value exceeds the specified cutoff value (e.g. negative values).


In [0]:
telco_customer_casted_df.select("TotalCharges", "tenure").display()


In [0]:
from pyspark.sql.functions import col

# Remove customers with negative TotalCharges
TotalCharges_cutoff = 0

# Use .filter method and SQL col() function
telco_no_outliers_df = telco_customer_casted_df.filter(
    (col("TotalCharges") > TotalCharges_cutoff) | (col("TotalCharges").isNull())  # Keep Nulls
)


### Removing outliers from PaymentMethod

- Identify the two lowest occurrence groups in the `PaymentMethod` column and calculating the total count and average `MonthlyCharges` for each group.
- Removing customers from the identified low occurrence groups in the `PaymentMethod` column to filter out outliers.
- Create a new dataframe `telco_filtered_df` containing the filtered data.
- Comparing the count of records before and after by dividing the count of `telco_casted_full_df` and `telco_no_outliers_df` dataframe removing outliers and then materializing the resulting dataframe as a new table.


In [0]:
from pyspark.sql.functions import col, count, avg

# Identify 2 lowest group occurrences
group_var = "PaymentMethod"
stats_df = telco_no_outliers_df.groupBy(group_var) \
    .agg(count("*").alias("Total"), \
         avg("MonthlyCharges").alias("MonthlyCharges")) \
    .orderBy(col("Total").desc())

# Display
display(stats_df)


In [0]:
# Gather 2 lowest groups name assuming count threshold is below 20% of full dataset and monthly charges <20$
N = telco_no_outliers_df.count()  # total count
lower_groups = [elem[group_var] for elem in stats_df.head(2) if elem["Total"]/N < 0.2 and elem["MonthlyCharges"] < 20]

print(f"Removing groups: {', '.join(lower_groups)}")


In [0]:
# Filter/Remove listings from these low occurrence groups while keeping null occurrences
telco_no_outliers_df = telco_no_outliers_df.filter(
    ~col(group_var).isin(lower_groups) | col(group_var).isNull()
)


In [0]:
# Count/Compare datasets before/after removing outliers
print(f"Count – Before: {telco_customer_casted_df.count()} / After: {telco_no_outliers_df.count()}")


In [0]:
# Materialize/Snap table [OPTIONAL/for instructor only]
telco_no_outliers_df.write.mode("overwrite").saveAsTable(telco_customer_full_silver)


### Handling Missing Values

To handle missing values in dataset we need to identify columns with high percentages of missing data and drop those columns. Then, it removes rows with missing values. Numeric columns are imputed with 0, and string columns are imputed with 'N/A'. Overall, the code demonstrates a comprehensive approach to handling missing values in the dataset.

---

### Delete Columns

- Create a DataFrame called `missing_df` to count the missing values per column in the `telco_no_outliers_df` dataset.

- The `missing_df` DataFrame is then transposed for better readability using the TransposeDF function, which allows for easier analysis of missing values.


In [0]:
from pyspark.sql.functions import col, when, count, concat_ws, collect_list

def calculate_missing(input_df, show=True):
    """
    Helper function to calculate and display missing data
    """

    # First get count of missing values per column to get a singleton row DF
    missing_df_ = input_df.select([
        count(
            when(
                col(c).contains('None') |
                col(c).contains('NULL') |
                (col(c) == '') |
                col(c).isNull(), c
            )
        ).alias(c) for c in input_df.columns
    ])

    # Transpose for better readability
    def TransposeDF(df, columns, pivotCol):
        """Helper function to transpose spark dataframe"""
        columnsValue = list(map(lambda x: str("") + str(x) + str(","), columns))
        stackCols = ','.join([x for x in columnsValue])
        df_1 = df.selectExpr(pivotCol, "stack(" + str(len(columns)) + "," + stackCols + ")") \
            .select(pivotCol, "col0", "col1") \
            .withColumnRenamed("col0", pivotCol)
        final_df = df_1.groupBy(pivotCol).pivot(pivotCol).agg(concat_ws("", collect_list(col("col1"))))
        return final_df

    missing_df_out_T = TransposeDF(
        spark.createDataFrame([{"Column": "Number of Missing Values"}]).join(missing_df_),
        missing_df_.columns,
        "Column"
    ).withColumn("Number of Missing Values", col("Number of Missing Values").cast("long"))

    if show:
        display(missing_df_out_T.orderBy("Number of Missing Values", ascending=False))

    return missing_df_out_T

# Ejecutar la función para calcular y mostrar missing_df
missing_df = calculate_missing(telco_no_outliers_df)


### Drop columns with more than x% of missing rows

Columns with more than 60% missing data are identified and stored in the `to_drop_missing` list, and these columns are subsequently dropped from the `telco_no_outliers_df` dataset.


In [0]:
# Drop if column has more than 60% missing data
per_thresh = 0.6
N = telco_no_outliers_df.count()  # total count

to_drop_missing = [
    x.asDict()['Column'] for x in missing_df.select("Column")
    .where(col("Number of Missing Values") / N >= per_thresh)
    .collect()
]

print(f"Dropping columns {to_drop_missing} for more than {per_thresh * 100}% missing data")
telco_no_missing_df = telco_no_outliers_df.drop(*to_drop_missing)

# display(telco_no_missing_df)


### Drop rows containing specific numbers of missing columns/fields

Rows with more than 1/4 of the columns missing values are dropped using the `na.drop()` and the remaining missing values in numeric columns are imputed with 0, while missing values in string columns are imputed with 'N/A'.


In [0]:
n_cols = len(telco_no_missing_df.columns)
telco_no_missing_df = telco_no_missing_df.na.drop(how='any', thresh=round(n_cols / 4))  # Drop rows where at least half values are missing, how='all' can also be used


In [0]:
# Count/Compare datasets before/after removing missing
print(f"Count – Before: {telco_no_outliers_df.count()} / After: {telco_no_missing_df.count()}")


### Impute Missing Data

Replace missing values with a specified replacement value.

- The `num_cols` and `string_cols` lists are created to identify numeric and string columns in the dataset, respectively.

- Finally, missing values in the numeric and string columns are imputed with appropriate values using the `na.fill()`, resulting in the `telco_imputed_df` dataset.


### Replace numeric missing with constant/0

NOTE: not applicable in this dataset's case


In [0]:
from pyspark.sql.types import DoubleType, IntegerType

# Get a list of numeric columns
num_cols = [c.name for c in telco_no_missing_df.schema.fields if (c.dataType == DoubleType() or c.dataType == IntegerType())]


In [0]:
# Impute
# telco_imputed_df = telco_no_missing_df.na.fill(value=0, subset=num_cols)


### Replace boolean missing with `False`


In [0]:
from pyspark.sql.types import BooleanType

# Get a list of boolean columns
bool_cols = [c.name for c in telco_no_missing_df.schema.fields if c.dataType == BooleanType()]

# Impute
telco_imputed_df = telco_no_missing_df.na.fill(value=False, subset=bool_cols)


### Replace string missing with `No`

All string cols except `gender`, `Contract` and `PaymentMethod`


In [0]:
from pyspark.sql.types import StringType

# Get list of string cols
to_exclude = ["customerID", "gender", "Contract", "PaymentMethod"]
string_cols = [c.name for c in telco_no_missing_df.drop(*to_exclude).schema.fields if c.dataType == StringType()]

# Impute
telco_imputed_df = telco_imputed_df.na.fill(value='No', subset=string_cols)


In [0]:
# Compare missing stats again
calculate_missing(telco_imputed_df)

### Encoding Categorical Features

In this section, we will one-hot encode categorical/string features using Spark MLlib's `OneHotEncoder` estimator.

If you are unfamiliar with one-hot encoding, there's a description below. If you're already familiar, you can skip ahead to the **One-hot encoding in Spark MLlib** section toward the bottom of the cell.

---

#### Categorical features in machine learning

Many machine learning algorithms are not able to accept categorical features as inputs. As a result, data scientists and machine learning engineers need to determine how to handle them.

An easy solution would be to remove the categorical features from the feature set. While this is quick, **you are removing potentially predictive information** — so this usually isn’t the best strategy.

Other options include ways to represent categorical features as numeric features. A few common options are:
1. **One-hot encoding**: create dummy/binary variables for each category.
2. **Target/label encoding**: replace each category value with a value that represents the target variable (e.g. replace a specific category value with the mean of the target variable for rows with that category value).
3. **Embeddings**: use/create a vector-representation of meaningful words in each category's value.

Each of these options can be really useful in different scenarios. We're going to focus on one-hot encoding here.

#### One-hot encoding basics

One-hot encoding creates a binary/dummy feature for each category in each categorical feature.

In the example below, the feature `Animal` is split into three binary features — one for each value in `Animal`. Each binary feature's value is equal to 1 if its respective category value is present in `Animal` for each row. If its category value is not present in the row, the binary feature's value will be 0.


#### One-hot encoding in Spark MLlib

Even if you understand one-hot encoding, it's important to learn how to perform it using Spark MLlib.

To one-hot encode categorical features in Spark MLlib, we are going to use two classes: the `StringIndexer` class and the `OneHotEncoder` class.

- The `StringIndexer` class indexes string-type columns to a numerical index. Each unique value in the string-type column is mapped to a unique integer.
- The `OneHotEncoder` class accepts indexed columns and converts them to a one-hot encoded vector-type feature.

---

#### Applying the `StringIndexer -> OneHotEncoder -> VectorAssembler` workflow

First, we'll need to index the categorical features of the DataFrame. `StringIndexer` takes a few arguments:
1. A list of categorical columns to index.
2. A list names for the indexed columns being created.
3. Directions for how to handle new categories when transforming data.

Because `StringIndexer` has to learn which categories are present before indexing, it's an **estimator** — remember that means we need to call its `fit` method. Its result can then be used to transform our data.



In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType

# Crear sesión de Spark (si no está activa)
spark = SparkSession.builder.getOrCreate()

# Datos simulados
data = [
    ("Month-to-month",),
    ("One year",),
    ("Two year",),
    ("Month-to-month",),
    ("Two year",),
]

# Definir esquema
schema = StructType([StructField("Contract", StringType(), True)])

# Crear DataFrame
telco_imputed_df = spark.createDataFrame(data, schema)

In [0]:
sample_df = telco_imputed_df.select("Contract").distinct()
sample_df.show()


In [0]:
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col

# StringIndexer
string_cols = ["Contract"]
index_cols = [column + "_index" for column in string_cols]

string_indexer = StringIndexer(inputCols=string_cols, outputCols=index_cols, handleInvalid="skip")
string_indexer_model = string_indexer.fit(sample_df)
indexed_df = string_indexer_model.transform(sample_df)

indexed_df.show()

In [0]:
# from pyspark.ml.feature import StringIndexer
# from pyspark.sql.functions import col

# # StringIndexer
# string_cols = ["Contract"]
# index_cols = [column + "_index" for column in string_cols]

# string_indexer = StringIndexer(inputCols=string_cols, outputCols=index_cols, handleInvalid="skip")
# string_indexer_model = string_indexer.fit(sample_df)
# indexed_df = string_indexer_model.transform(sample_df)

# indexed_df.show()

# arriba falla este no
from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank

# Crear índice categórico manual
window_spec = Window.orderBy("Contract")
indexed_df = sample_df.withColumn("Contract_index", dense_rank().over(window_spec) - 1)

indexed_df.show()

Once our data has been indexed, we are ready to use the `OneHotEncoder` estimator.

💡 **Hint:** Look at the `OneHotEncoder` [documentation](https://spark.apache.org/docs/latest/ml-features.html#onehotencoder) and our previous Spark MLlib workflows that use estimators for guidance.


In [0]:
from pyspark.ml.feature import OneHotEncoder

# Create a list of one-hot encoded feature names
ohe_cols = [column + "_ohe" for column in string_cols]

# Instantiate the OneHotEncoder with the column lists
ohe = OneHotEncoder(inputCols=index_cols, outputCols=ohe_cols, handleInvalid="keep")

# Fit the OneHotEncoder on the indexed data
ohe_model = ohe.fit(indexed_df)

# Transform indexed_df using the ohe_model
ohe_df = ohe_model.transform(indexed_df)
ohe_df.show()


In [0]:
from pyspark.ml.feature import VectorAssembler

selected_ohe_cols = ["Contract_ohe"]

# Use VectorAssembler to assemble the selected one-hot encoded columns into a dense vector
assembler = VectorAssembler(inputCols=selected_ohe_cols, outputCol="features")
result_df_dense = assembler.transform(ohe_df)

# Select relevant columns for display
result_df_display = result_df_dense.select("Contract", "features")
result_df_display.show(truncate=False)


### Apply pre-existing embeddings to categorical/discrete features

Let's bin `tenure` to convert the discrete data into bins/categories format for further analysis and modeling.


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Crear sesión de Spark (si no está activa)
spark = SparkSession.builder.getOrCreate()

# Datos simulados
data = [
    (1, "Month-to-month"),
    (5, "Month-to-month"),
    (12, "One year"),
    (24, "Two year"),
    (36, "Two year"),
    (60, "Two year")
]

# Definir esquema
schema = StructType([
    StructField("tenure", IntegerType(), True),
    StructField("Contract", StringType(), True)
])

# Crear DataFrame
telco_imputed_df = spark.createDataFrame(data, schema)

# Mostrar
telco_imputed_df.select("tenure").show()


In [0]:
column_to_bin = "tenure"
display(telco_imputed_df.select(column_to_bin))


In [0]:
from pyspark.ml.feature import Bucketizer
from pyspark.sql.functions import col

# Specify bin ranges and column to bin
bucketizer = Bucketizer(
    splits=[0, 24, 48, float('Inf')],
    inputCol=column_to_bin,
    outputCol=f"{column_to_bin}_bins"
)

# Apply the bucketizer to the DataFrame
bins_df = bucketizer.transform(telco_imputed_df.select(column_to_bin))

# Recast bin numbers to integer
bins_df = bins_df.withColumn(f"{column_to_bin}_bins", col(f"{column_to_bin}_bins").cast("integer"))

# Display the result
display(bins_df)


Map back to human-readable embedding scores


In [0]:
bins_embedded_df = (
    bins_df.withColumn(f"{column_to_bin}_embedded", col(f"{column_to_bin}_bins").cast(StringType()))
    .replace(to_replace={
        "0": "<2y",
        "1": "2-4y",
        "2": ">4y"
    }, subset=[f"{column_to_bin}_embedded"])
)

display(bins_embedded_df)


### Ordered Indexing

Perform ordered indexing as an alternative categorical feature preparation for random forest modeling.

Some categoricals are in fact `ordinal` and thus may require additional/manual encoding.


In [0]:
ordinal_cat = "Contract"
telco_imputed_df.select(ordinal_cat).distinct().show(truncate=False)

In [0]:
# Define Ordinal (category:index) map/dict
ordered_list = [
    "Month-to-month",
    "One year",
    "Two year"
]

ordinal_dict = {category: f"{index+1}" for index, category in enumerate(ordered_list)}
display(ordinal_dict)


In [0]:
# Create a new column with ordered indexing
from pyspark.sql.functions import expr

ordinal_df = (
    telco_imputed_df
    .withColumn(f"{ordinal_cat}_ord", col(ordinal_cat))  # Duplicate column
    .replace(to_replace=ordinal_dict, subset=[f"{ordinal_cat}_ord"])  # Map to ordinal values
    .withColumn(f"{ordinal_cat}_ord", col(f"{ordinal_cat}_ord").cast('int'))  # Cast to integer
)

display(ordinal_df.select(ordinal_cat, f"{ordinal_cat}_ord"))


## Splitting Data (for Cross-Validation)

Split modeling data into a train-test-holdout split as part of a modeling process.

In this section, we will perform the best-practice workflow for a train-test split using the Spark DataFrame API.

Recall that due to things like changing cluster configurations and data partitioning, it can be difficult to ensure a reproducible train-test split. As a result, we recommend:

1. Split the data using the same random seed
2. Write out the train and test DataFrames

💡 **Hint:** Check out the `randomSplit` [documentation](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.randomSplit.html).


In [0]:
# Split with 80 percent of the data in train_df and 20 percent of the data in test_df
train_df, test_df = telco_imputed_df.randomSplit([.8, .2], seed=42)


In [0]:
# Materialize (OPTIONAL)
train_df.write.mode("overwrite").option("overwriteSchema", True).saveAsTable(f"{DA.catalog_name}.{DA.schema_name}.telco_customers_train")
test_df.write.mode("overwrite").option("overwriteSchema", True).saveAsTable(f"{DA.catalog_name}.{DA.schema_name}.telco_customers_baseline")


## Standardize Features in a Training Set

For sake of example, we'll pick a column without missing data (e.g. `MonthlyCharges`)


In [0]:
from pyspark.ml.feature import StandardScaler, RobustScaler, VectorAssembler

# Columnas a escalar
num_cols_to_scale = ["MonthlyCharges"]  # num_cols

# Crear ensamblador
assembler = VectorAssembler().setInputCols(num_cols_to_scale).setOutputCol("numerical_assembled")

# Transformar train y test
train_assembled_df = assembler.transform(train_df.select(*num_cols_to_scale))
test_assembled_df = assembler.transform(test_df.select(*num_cols_to_scale))

# Definir scaler y ajustar en conjunto de entrenamiento
scaler = RobustScaler(inputCol="numerical_assembled", outputCol="numerical_scaled")
scaler_fitted = scaler.fit(train_assembled_df)

# Aplicar a ambos conjuntos
train_scaled_df = scaler_fitted.transform(train_assembled_df)
test_scaled_df = scaler_fitted.transform(test_assembled_df)


In [0]:
print("Peek at Training set")
train_scaled_df.show(5)

## Impute categorical missing values with the mode value using sparkml

How to handle missing data only at training time and bake as part of inference pipeline to avoid data leakage and ensure that observation with missing data is used for training.


In [0]:
categorical_cols_to_impute = ["PaymentMethod"]  # string_cols


Index categoricals first as Imputer doesn't handle categoricals directly

In [0]:
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import  col

# Index categorical columns using StringIndexer
cat_index_cols = [column + "_index" for column in categorical_cols_to_impute]
cat_indexer = StringIndexer(
    inputCols=categorical_cols_to_impute,
    outputCols=cat_index_cols,
    handleInvalid="keep"
)

# Fit on training set
cat_indexer_model = cat_indexer.fit(train_df.select(categorical_cols_to_impute))


In [0]:
# Transform both train & test set using the fitted StringIndexer model
cat_indexed_train_df = cat_indexer_model.transform(train_df.select(*categorical_cols_to_impute))
cat_indexed_test_df = cat_indexer_model.transform(test_df.select(*categorical_cols_to_impute))

# display(cat_indexed_train_df)


In [0]:
cat_indexed_train_df.display()


The `StringIndexer` will create a new label (e.g. `4`) for missing when setting the `handleInvalid` flag to `keep`, so it's important to keep track/revert indexes values back to `null` if we want to impute them, otherwise `null` will be treated as their own/separate category automatically.

Alternatively for imputing categorical/strings, we can use `.fillna()` method by providing the `mode` value manually (as described above).


In [0]:
# Revert indexes to `null` for missing categories
for c in categorical_cols_to_impute:
    cat_indexed_train_df = cat_indexed_train_df.withColumn(
        f"{c}_index",
        when(col(c).isNull(), None).otherwise(col(f"{c}_index"))
    )
    cat_indexed_test_df = cat_indexed_test_df.withColumn(
        f"{c}_index",
        when(col(c).isNull(), None).otherwise(col(f"{c}_index"))
    )


In [0]:
cat_indexed_train_df.display()

Fit the imputer on indexed categoricals


In [0]:
from pyspark.ml.feature import Imputer

# Define 'mode' imputer
output_cat_index_cols_imputed = [col + '_imputed' for col in cat_index_cols]
mode_imputer = Imputer(
    inputCols=cat_index_cols,
    outputCols=output_cat_index_cols_imputed,
    strategy="mode"
)

# Fit on training_df
mode_imputer_fitted = mode_imputer.fit(cat_indexed_train_df)

In [0]:
cat_indexed_train_imputed_df = mode_imputer_fitted.transform(cat_indexed_train_df)
cat_indexed_test_imputed_df = mode_imputer_fitted.transform(cat_indexed_test_df)